In [1]:
import os
import shutil
import contextlib
import pathlib
import json

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import run
import environment
import evolve
import encode
import plot

In [2]:
run1_results_dir = 'results/nearest_simple_sbx_100/'

run1_vision = environment.NearestFoodParticleVision()
run1_genome_size = [run1_vision.organism_input_shape, 5, 2]

fitness = evolve.EnergyFitness()

arguments = {
    'start_organism_number': 40,
    'width': 12,
    'height': 12,
    'iterations': 80 * 100,
    'generation_time': 80,
    'organism_size': 0.12,
    'food_size': 0.05,
    'organism_vision_range': 1,
    'results_dir': run1_results_dir,
    'vision': run1_vision,
    'food_energy': 2,
    'food_appearance_number_rate': 0.7,
    'encoding': encode.RealValued(),
    'selection': evolve.TruncationSelection(fitness=fitness, n=30),
    'crossover': evolve.SBXCrossover(n=2),
    'mutation': evolve.GaussianMutation(mu=0, sigma=0.1, p=0.2),
    'elitism': 10,
    'genome_size': run1_genome_size,
    'food_particles_at_start': 100,
    'remove_dead_organisms': False,
}

# run1_params_file = os.path.join(run1_results_dir, 'data.json')

In [3]:
if os.path.exists(run1_results_dir):
    ans = input('Previous results folder exists. Do you want to delete it?')
    if ans == 'y':
        shutil.rmtree(run1_results_dir)

with contextlib.suppress(Exception):
    os.mkdir(run1_results_dir)
    
# with open(run1_params_file, 'w') as file:
#     json.dump(arguments, file)

run1 = run.OrganismsSimpleEnvironmentRunTool(**arguments)
run1.run()

Simulating organisms: 100%|██████████| 8000/8000 [28:52<00:00,  4.62it/s, Number of organisms=40, Gen=99]


In [4]:
org_loc = pq.read_table(os.path.join(run1_results_dir,
                                     'organisms_locations')).to_pandas()
org_stat = pq.read_table(os.path.join(run1_results_dir,
                                      'organisms_stats')).to_pandas()
food_loc = pq.read_table(os.path.join(run1_results_dir,
                                      'food_locations')).to_pandas()
food_stat = pq.read_table(os.path.join(run1_results_dir,
                                       'eaten_food')).to_pandas()


org_loc.reset_index(drop=True, inplace=True)
org_stat.reset_index(drop=True, inplace=True)
food_loc.reset_index(drop=True, inplace=True)
food_stat.reset_index(drop=True, inplace=True)

In [5]:
org_loc.describe()

,index,x,y,theta,iteration,generation,v,a,energy,age
count,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000,320000.000000
mean,1502.753500,5.763285,6.896729,-3.108934,3999.500000,49.500000,-0.081636,-0.046219,-4.048194,59.312500
std,866.231134,3.595239,3.664843,5.042462,2309.404667,28.866115,0.030622,0.018252,3.518383,41.534717
min,0.000000,0.000000,0.000000,-32.000000,0.000000,0.000000,-0.100000,-0.050000,-12.800000,0.000000
25%,752.000000,2.706290,3.881764,-4.640061,1999.750000,24.750000,-0.093427,-0.050000,-6.160000,26.000000
50%,1502.500000,5.745668,7.084459,-2.173767,3999.500000,49.500000,-0.084825,-0.050000,-3.280000,53.000000
75%,2252.250000,8.761339,10.197920,-0.800000,5999.250000,74.250000,-0.083333,-0.050000,-1.360000,79.000000
max,3009.000000,12.000000,12.000000,32.000000,7999.000000,99.000000,0.100000,0.050000,11.360000,160.000000


In [6]:
org_stat.describe()

,id,age,iteration,energy
count,320000.000000,320000.000000,320000.000000,320000.000000
mean,1502.753500,59.312500,3999.500000,-4.048194
std,866.231134,41.534717,2309.404667,3.518383
min,0.000000,0.000000,0.000000,-12.800000
25%,752.000000,26.000000,1999.750000,-6.160000
50%,1502.500000,53.000000,3999.500000,-3.280000
75%,2252.250000,79.000000,5999.250000,-1.360000
max,3009.000000,160.000000,7999.000000,11.360000


In [7]:
food_stat.describe()

,organism_id,food_location,energy_taken,iteration
count,5180.000000,5180.000000,5180.0,5180.000000
mean,1492.236680,6.037994,2.0,3962.544788
std,855.534853,3.446265,0.0,2283.353096
min,1.000000,0.000846,2.0,0.000000
25%,782.000000,3.079138,2.0,2103.000000
50%,1470.000000,5.956656,2.0,3908.500000
75%,2195.000000,8.974517,2.0,5850.000000
max,3007.000000,11.999827,2.0,7999.000000


In [8]:
org_loc.groupby('generation')[['energy', 'age']].describe()

energy                                                        age   
             count      mean       std    min   25%   50%   75%   max   count   
generation                                                                      
0           3200.0 -2.093125  1.843431  -6.40 -3.44 -1.92 -0.72  2.64  3200.0  \
1           3200.0 -3.616250  3.138840 -12.80 -5.68 -3.04 -1.12  3.44  3200.0   
2           3200.0 -4.159375  3.490853 -12.72 -6.32 -3.20 -1.44  1.84  3200.0   
3           3200.0 -4.031875  3.555036 -12.72 -6.24 -3.36 -1.36  4.80  3200.0   
4           3200.0 -4.076875  3.645037 -12.72 -6.34 -3.28 -1.28  4.48  3200.0   
...            ...       ...       ...    ...   ...   ...   ...   ...     ...   
95          3200.0 -4.130625  3.247304 -12.72 -6.16 -3.36 -1.60  2.32  3200.0   
96          3200.0 -4.206875  3.452662 -12.72 -6.16 -3.44 -1.52  1.84  3200.0   
97          3200.0 -3.708125  3.657614 -12.72 -6.24 -3.12 -1.04  3.92  3200.0   
98          3200.0 -4.168125  3.375136 -12.72 -6.16 -3.52 -1.52  2.40  3200.0   
99          3200.0 -4.241875  3.454863 -12.72 -6.32 -3.68 -1.44  2.72  3200.0   

                                                              
             mean        std  min    25%   50%    75%    max  
generation                                                    
0           40.50  23.095815  1.0  20.75  40.5  60.25   80.0  
1           59.75  41.999866  0.0  26.00  53.0  79.50  160.0  
2           59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
3           59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
4           59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
...           ...        ...  ...    ...   ...    ...    ...  
95          59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
96          59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
97          59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
98          59.50  41.638826  0.0  26.00  53.0  79.25  159.0  
99          59.50  41.638826  0.0  26.00  53.0  79.25  159.0  

[100 rows x 16 columns]

In [9]:
# frames_dir = os.path.join(run1_results_dir, 'frames')
# frames_dir = pathlib.Path(run1_results_dir, 'frames')
# frames_dir.mkdir()
# plot.create_frames(org_loc, food_loc, arguments['width'], arguments['height'], arguments['organism_size'], arguments['food_size'], frames_dir)

In [10]:
# plot.generate_video(
#     os.path.join(run1_results_dir, 'frames/'),
#     framerate=24,
#     output=f'{run1_results_dir}evolution.mp4',
# )

# Inspecting results

In [11]:
from plotly import express as px
import plotly.graph_objs as go

In [12]:
WINDOW = 5

moving_age_mean = org_loc[[
    'iteration', 'age'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['iteration', 'age'
                          ]].groupby('iteration').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[[
    'iteration', 'energy'
]].groupby('iteration').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)
mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW}',
                    mode='lines',
                    legendgroup='Age')
ma_energy = go.Scatter(x=x,
                       y=moving_energy_mean['energy'],
                       name=f'Moving Energy Average {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')
mm_energy = go.Scatter(x=x,
                       y=moving_energy_max['energy'],
                       name=f'Moving Energy Max {WINDOW}',
                       mode='lines',
                       legendgroup='Energy')

layout = go.Layout(title='Evolution Age/Energy Statistics',
                   xaxis=dict(title='Time'))

fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [13]:
WINDOW = 3

moving_age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean().rolling(WINDOW).mean()

moving_age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max().rolling(WINDOW).mean()

moving_energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean().rolling(WINDOW).mean()

moving_energy_max = org_loc[['generation', 'energy']].groupby('generation').max().rolling(WINDOW).mean()

x = np.arange(len(moving_age_max))

ma_age = go.Scatter(
    x=x,
    y=moving_age_mean['age'],
    name=f'Moving Age Average {WINDOW}',
    mode='lines',
    legendgroup='Age',
)

mm_age = go.Scatter(x=x,
                    y=moving_age_max['age'],
                    name=f'Moving Age Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Age')

ma_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Max {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

mm_energy = go.Scatter(x=x,
                    y=moving_energy_mean['energy'],
                    name=f'Moving Energy Average {WINDOW=}',
                    mode='lines',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Age/Energy Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[ma_age, mm_age, mm_energy, ma_energy], layout=layout)
fig.show()

In [14]:
age_mean = org_loc[[
    'generation', 'age'
]].groupby('generation').mean()

age_max = org_loc[['generation', 'age'
                          ]].groupby('generation').max()

energy_mean = org_loc[['generation', 'energy']].groupby('generation').mean()

energy_max = org_loc[['generation', 'energy']].groupby('generation').max()

energy_max_ma = energy_max.rolling(5).mean()

x = np.arange(len(age_max))

age_avg = go.Scatter(
    x=x,
    y=age_mean['age'],
    name=f'Age Average',
    mode='lines',
    legendgroup='Age',
)

age_max = go.Scatter(x=x,
                    y=age_max['age'],
                    name=f'Age Max',
                    mode='lines',
                    legendgroup='Age')

energy_mean = go.Scatter(x=x,
                    y=energy_mean['energy'],
                    name=f'Energy Average',
                    mode='lines',
                    legendgroup='Energy')

energy_max = go.Scatter(x=x,
                    y=energy_max['energy'],
                    name=f'Energy Max',
                    mode='lines',
                    marker_color='rgba(255, 180, 0, 100)',
                    legendgroup='Energy')

energy_max_ma = go.Scatter(x=x,
                    y=energy_max_ma['energy'],
                    name=f'MA Energy Max',
                    mode='lines',
                    marker_color='rgba(255, 50, 0, 100)',
                    legendgroup='Energy')

layout = go.Layout(title='Evolution Energy Statistics',
                   xaxis=dict(title='Generation'))

fig = go.Figure(data=[energy_mean, energy_max, energy_max_ma], layout=layout)
# fig = go.Figure(data=[age_avg, age_max, energy_mean], layout=layout)

fig.show()

In [15]:
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])